<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/CNN_Experiments/CIFAR_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# python3 CIFAR_runner.py --eps 0.004 --rob 0 --lam 0.25 --opt NA --gpu 3 &
# python3 CIFAR_runner.py --eps 0.004 --rob 0 --lam 0.25 --opt VOGN --gpu 4 &
# python3 CIFAR_runner.py --eps 0.004 --rob 0 --lam 0.25 --opt SWAG --gpu 5 &

dict = {'eps': 0.004, 'rob': 0, 'lam': 0.25, 'opt': 'NA', 'gpu': '3'}

eps = dict['eps']
lam = dict['lam']
rob = dict['rob']
optim = dict['opt']
gpu = dict['gpu']
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

In [ ]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/Rayleigh_Experiments/'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import BayesKeras
import BayesKeras.optimizers as optimizers


(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

augment_size = 40000
image_generator = ImageDataGenerator(
    rotation_range=10,
    zoom_range = 0.075, 
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    vertical_flip=False, 
    data_format="channels_last",
    zca_whitening=False)
# fit data for zca whitening
image_generator.fit(X_train, augment=True)
# get transformed images
randidx = np.random.randint(50000, size=augment_size)
x_augmented = X_train[randidx].copy()
y_augmented = y_train[randidx].copy()
x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size),
                            batch_size=augment_size, shuffle=False).next()[0]
# append augmented data to trainset
X_train = np.concatenate((X_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

#X_train = X_train[0:10000]
#y_train = y_train[0:10000]

model_type = "small"
if(model_type == "VGG8"):
    model = Sequential()
    #tf.keras.layers.GaussianNoise(stddev, **kwargs)
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='elu', input_shape=(32,32,3)))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='elu'))
    #model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='elu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='elu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='valid', activation='elu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='valid', activation='elu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation = 'softmax'))

elif(model_type == "small"):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=(4, 4), activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(filters=32, kernel_size=(4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation = 'softmax'))

elif(model_type == "mini"):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(4, 4), activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(filters=32, kernel_size=(4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation = 'softmax'))

elif(model_type == "medium"): 
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(filters=32, kernel_size=(4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(filters=64, kernel_size=(4, 4), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation = 'softmax'))

elif(model_type == "large"):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(10, activation = 'softmax'))

lr = 1
print("Got flag: %s"%(optim))
if(optim == 'VOGN'):
#    learning_rate = 0.35*lr; decay=0.075
    learning_rate = 0.25*lr; decay=0.025
    #learning_rate = 0.05*lr; decay=0.0
    opt = optimizers.VariationalOnlineGuassNewton()
elif(optim == 'BBB'):
    learning_rate = 0.65*lr; decay=0.0
    opt = optimizers.BayesByBackprop()
elif(optim == 'SWAG'):
#    learning_rate = 0.0125*lr; decay=0.025
    learning_rate = 0.015*lr; decay=0.0
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'NA'):
    learning_rate = 0.00025*lr; decay=0.025
    opt = optimizers.NoisyAdam()
elif(optim == 'SGD'):
    learning_rate = 0.05*lr; decay=0.1
    opt = optimizers.StochasticGradientDescent()
# Compile the model to train with Bayesian inference
if(rob == 0 or rob == 3 or rob == 4):
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
else:
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss

inf = 2.5
#learning_rate *= 1.5

bayes_model = opt.compile(model, loss_fn=loss, epochs=45, learning_rate=learning_rate, batch_size=128, input_noise=0.0,
                          decay=decay, robust_train=rob, epsilon=eps, rob_lam=lam, inflate_prior=inf, log_path="%s_%s_Posterior_%s.log"%(optim, model_type, rob))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


170508288/170498071 [==============================] - 3s 0us/step
Got flag: NA
This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
BayesKeras: Using implicit prior
(4, 4, 3, 16) 0.22821773229381923
(4, 4, 16, 32) 0.09882117688026186
(5408, 100) 0.02150065362980567
(100, 10) 0.15811388300841897
BayesKeras: Using implicit prior
(4, 4, 3, 16) 0.22821773229381923
(4, 4, 16, 32) 0.09882117688026186
(5408, 100) 0.02150065362980567
(100, 10) 0.15811388300841897


In [ ]:
# Train the model on your data
bayes_model.train(X_train, y_train, X_test, y_test)

# Save your approxiate Bayesian posterior
bayes_model.save(experiment + "%s_%s_Posterior_%s"%(optim, model_type, rob))

  0%|          | 0/704 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
100%|██████████| 704/704 [04:22<00:00,  2.68it/s]


Epoch 1, loss: 1.840, acc: 0.331, val_loss: 1.439, val_acc: 0.491


100%|██████████| 704/704 [04:23<00:00,  2.67it/s]


Epoch 2, loss: 1.353, acc: 0.518, val_loss: 1.261, val_acc: 0.549


100%|██████████| 704/704 [03:33<00:00,  3.29it/s]


...

Epoch 43, loss: 0.566, acc: 0.812, val_loss: 1.622, val_acc: 0.643


100%|██████████| 704/704 [03:31<00:00,  3.33it/s]


Epoch 44, loss: 0.566, acc: 0.813, val_loss: 1.565, val_acc: 0.653


100%|██████████| 704/704 [04:22<00:00,  2.68it/s]


Epoch 45, loss: 0.572, acc: 0.810, val_loss: 1.513, val_acc: 0.651
